# Building a RAG application for Youtube


Let's start by loading the environment variables.

In [22]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


## Setting up the model
Define the LLM model that we'll use as part of the workflow.

In [23]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

 Test the model

In [26]:
model.invoke("Who won the Tokyo Olympic Badminton Men Single during the COVID-19 pandemic?")

AIMessage(content="During the COVID-19 pandemic, Viktor Axelsen of Denmark won the gold medal in the Men's Singles badminton event at the Tokyo Olympics in 2021.", response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 24, 'total_tokens': 58}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e5435dcf-f819-4ed0-b684-254eaa6bbd13-0')

In this example, use a simple `StrOutputParser` to extract the asnwer as string

In [27]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Who won the Tokyo Olympic Badminton Men Single during the COVID-19 pandemic?")

"Viktor Axelsen of Denmark won the gold medal in the Men's Singles badminton event at the Tokyo Olympics in 2021 during the COVID-19 pandemic."

## Introducing prompt templates

We use prompt templates to give the model the necessary context and the question. [Prompt templates](https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/) are an efficient way to define and reuse prompts.

In [34]:
from langchain_core.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. 
If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format (context="Jane's brother is John", question="Who is John's brother?")

'Human: \nAnswer the question based on the context below. \nIf you can\'t answer the question, reply "I don\'t know".\n\nContext: Jane\'s brother is John\n\nQuestion: Who is John\'s brother?\n'

We can now chain the prompt with the model and the output parser.

In [40]:
chain = prompt | model | parser
chain.invoke({"context": "Jane's brother is John", 
              "question":"Who is Jane's brother?"})

'John'